#### imports

In [7]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

# load the data

In [8]:
#!wget -O loan_train.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/loan_train.csv

#### load csv into dataframe

In [9]:
df = pd.read_csv('loan_train.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,0,0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,2,2,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,3,3,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,4,4,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,6,6,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male


In [10]:
df.shape

(346, 10)

# clean the data

#### remove unamed

In [11]:
df = df.drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1)
df.head()

,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,PAIDOFF,1000,30,9/8/2016,10/7/2016,45,High School or Below,male
1,PAIDOFF,1000,30,9/8/2016,10/7/2016,33,Bechalor,female
2,PAIDOFF,1000,15,9/8/2016,9/22/2016,27,college,male
3,PAIDOFF,1000,30,9/9/2016,10/8/2016,28,college,female
4,PAIDOFF,1000,30,9/9/2016,10/8/2016,29,college,male


#### convert date

In [12]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,male
1,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,female
2,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,male
3,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,female
4,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,male


#### loan_status column is what we want to predict (target)

In [13]:
print(df['loan_status'].unique())
print(df['loan_status'].value_counts())

['PAIDOFF' 'COLLECTION']
PAIDOFF       260
COLLECTION     86
Name: loan_status, dtype: int64


#### convert male to 0 and female to 1

In [14]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

,loan_status,Principal,terms,effective_date,due_date,age,education,Gender
0,PAIDOFF,1000,30,2016-09-08,2016-10-07,45,High School or Below,0
1,PAIDOFF,1000,30,2016-09-08,2016-10-07,33,Bechalor,1
2,PAIDOFF,1000,15,2016-09-08,2016-09-22,27,college,0
3,PAIDOFF,1000,30,2016-09-09,2016-10-08,28,college,1
4,PAIDOFF,1000,30,2016-09-09,2016-10-08,29,college,0


#### set a column for categorical variable with 0 or 1

In [15]:
print(df['education'].unique())
print(df['education'].value_counts())

['High School or Below' 'Bechalor' 'college' 'Master or Above']
High School or Below    151
college                 149
Bechalor                 44
Master or Above           2
Name: education, dtype: int64


In [16]:
# keep non categorical columns
Feature = df[['Principal','terms','age','Gender']]
# set a column for each categorical value
Feature = pd.concat([Feature,pd.get_dummies(df['education'])], axis=1)
# drop "Master or above" column, too few values
Feature.drop(['Master or Above'], axis = 1,inplace=True)
Feature.head()

,Principal,terms,age,Gender,Bechalor,High School or Below,college
0,1000,30,45,0,0,1,0
1,1000,30,33,1,1,0,0
2,1000,15,27,0,0,0,1
3,1000,30,28,1,0,0,1
4,1000,30,29,0,0,0,1


#### set X and Y for classification

In [17]:
X = Feature
y = df['loan_status'].values

#### Normalize Data

In [18]:
X = preprocessing.StandardScaler().fit(X).transform(X)

# Classification

#### split the dataset

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (276, 7) (276,)
Test set: (70, 7) (70,)


## KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier

# train Model and Predict with k = 3
kNN_model = KNeighborsClassifier(n_neighbors=3)
kNN_model.fit(X_train, y_train)
kNN_model.score(X_test, y_test)

0.5571428571428572

In [21]:
# use GridSearchCV to get best hyperparameters for KNN
from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors' : np.arange(1, 20), 'metric' : ['euclidiean', 'manhattan']}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid.fit(X_train, y_train)

c:\Users\gerin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\gerin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\gerin\anaconda3\lib\site-packages\sklearn\neighbors\_base.py", line 1157, in fit
    return self._fit(X)
  File "c:\Users\gerin\anaconda3\lib\site-packages\sklearn\neighbors\_base.py", line 350, in _fit
    self._check_algorithm_metric()
  File "c:\Users\gerin\anaconda3\lib\site-packages\sklearn\neighbors\_base.py", line 332, in _check_algorithm_metric
    raise ValueError("Metric '%s' not valid. Use "
ValueError: Metric 'euclidiean' not valid. Use sorted(sklearn.neighbors.VALID_METRICS['brute']) to get valid options. Metric can also be a callable functio

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidiean', 'manhattan'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])})

In [22]:
print("best score for KNN", grid.best_score_)
print("best hypermarameters for KNN", grid.best_params_)

best score for KNN 0.7644805194805195
best hypermarameters for KNN {'metric': 'manhattan', 'n_neighbors': 17}


In [23]:
kNN_model = grid.best_estimator_
kNN_model.score(X_test, y_test)

0.7

## Decision tree

In [24]:
from sklearn.tree import DecisionTreeClassifier

DT_model = DecisionTreeClassifier()
DT_model.fit(X_train,y_train)
DT_model.score(X_test, y_test)

0.5285714285714286

In [25]:
param_grid = {'criterion' : ['gini', 'entropy'], 'max_depth' : np.arange(1, 20)}

grid = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])})

In [26]:
print("best score for Decision Tree", grid.best_score_)
print("best hypermarameters for Decision Tree", grid.best_params_)

best score for Decision Tree 0.7644805194805195
best hypermarameters for Decision Tree {'criterion': 'gini', 'max_depth': 1}


In [27]:
DT_model = grid.best_estimator_
DT_model.score(X_test, y_test)

0.7

## Support Vector Machine

In [28]:
from sklearn import svm

SVM_model = svm.SVC()
SVM_model.fit(X_train, y_train)
SVM_model.score(X_test, y_test)

0.7

In [29]:
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

grid = GridSearchCV(svm.SVC(), param_grid, cv=5)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']})

In [30]:
print("best score for SVM", grid.best_score_)
print("best hypermarameters for SVM", grid.best_params_)

best score for SVM 0.7644805194805195
best hypermarameters for SVM {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}


In [31]:
SVM_model = grid.best_estimator_
SVM_model.score(X_test, y_test)

0.7

## Logistic Regression

In [32]:
from sklearn.linear_model import LogisticRegression

LR_model = LogisticRegression()
LR_model.fit(X_train,y_train)
LR_model.score(X_test, y_test)

0.7

In [33]:
param_grid = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}

grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)

c:\Users\gerin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\gerin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\gerin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Users\gerin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
c:\Users\gerin\anaconda3\lib\site-packages\sklearn\model_selection\_v

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']})

In [34]:
print("best score for Linear Regression", grid.best_score_)
print("best hypermarameters for Linear Regression", grid.best_params_)

best score for Linear Regression 0.7644805194805195
best hypermarameters for Linear Regression {'C': 0.001, 'penalty': 'l2'}


In [35]:
LR_model = grid.best_estimator_
LR_model.score(X_test, y_test)

0.7

# Model accuracy

In [36]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [37]:
y_pred = kNN_model.predict(X_test)
print("KNN Jaccard :", jaccard_score(y_test, y_pred, average=None)[1])
print("KNN F1-score :", f1_score(y_test, y_pred, average='weighted'))

KNN Jaccard : 0.7
KNN F1-score : 0.5764705882352941


In [38]:
y_pred = DT_model.predict(X_test)
print("Decision Tree Jaccard :", jaccard_score(y_test, y_pred, average=None)[1])
print("Decision Tree F1-score :", f1_score(y_test, y_pred, average='weighted'))

Decision Tree Jaccard : 0.7
Decision Tree F1-score : 0.5764705882352941


In [39]:
y_pred = SVM_model.predict(X_test)
print("SVM Jaccard :", jaccard_score(y_test, y_pred, average=None)[1])
print("SVM F1-score :", f1_score(y_test, y_pred, average='weighted'))

SVM Jaccard : 0.7
SVM F1-score : 0.5764705882352941


In [40]:
y_pred = LR_model.predict(X_test)
print("Linear regression Jaccard :", jaccard_score(y_test, y_pred, average=None)[1])
print("Linear regression F1-score :", f1_score(y_test, y_pred, average='weighted'))

y_pred = LR_model.predict_proba(X_test)
print("Linear regression LogLoss :", log_loss(y_test, y_pred))

Linear regression Jaccard : 0.7
Linear regression F1-score : 0.5764705882352941
Linear regression LogLoss : 0.6209240963453982
